In [1]:
import requests
import lxml.html as lh
import pandas as pd
import bs4
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup

In [2]:
companies = pd.read_csv('List_of_companies.csv', encoding = "ISO-8859-1")

print(companies.dtypes)

companies['Security'] = companies['Security'].str.replace(r' ', '-')
companies['Security'] = companies['Security'].str.replace(r'.', '-')
companies['Security'] = companies['Security'].str.replace(r',', '-')
companies['Security'] = companies['Security'].str.lower()
print(len(companies))
companies

Symbol                   object
Security                 object
SEC filings              object
GICS Sector              object
GICS Sub-Industry        object
Headquarters Location    object
Date first added         object
CIK                       int64
Founded                  object
dtype: object
505


,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3m-company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",8/9/1976,66740,1902
1,ABT,abbott-laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",3/31/1964,1800,1888
2,ABBV,abbvie-inc-,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",12/31/2012,1551152,2013 (1888)
3,ABMD,abiomed-inc,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",5/31/2018,815094,1981
4,ACN,accenture-plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",7/6/2011,1467373,1989
5,ATVI,activision-blizzard,reports,Communication Services,Interactive Home Entertainment,"Santa Monica, California",8/31/2015,718877,2008
6,ADBE,adobe-inc-,reports,Information Technology,Application Software,"San Jose, California",5/5/1997,796343,1982
7,AMD,advanced-micro-devices-inc,reports,Information Technology,Semiconductors,"Santa Clara, California",3/20/2017,2488,1969
8,AAP,advance-auto-parts,reports,Consumer Discretionary,Automotive Retail,"Raleigh, North Carolina",7/9/2015,1158449,1932
9,AES,aes-corp,reports,Utilities,Independent Power Producers & Energy Traders,"Arlington, Virginia",10/2/1998,874761,1981


In [3]:
for n in range(10):
    currName = companies.at[n, 'Security']
    url='https://www1.salary.com/' + currName + '-Executive-Salaries.html'
    print(url)

https://www1.salary.com/3m-company-Executive-Salaries.html
https://www1.salary.com/abbott-laboratories-Executive-Salaries.html
https://www1.salary.com/abbvie-inc--Executive-Salaries.html
https://www1.salary.com/abiomed-inc-Executive-Salaries.html
https://www1.salary.com/accenture-plc-Executive-Salaries.html
https://www1.salary.com/activision-blizzard-Executive-Salaries.html
https://www1.salary.com/adobe-inc--Executive-Salaries.html
https://www1.salary.com/advanced-micro-devices-inc-Executive-Salaries.html
https://www1.salary.com/advance-auto-parts-Executive-Salaries.html
https://www1.salary.com/aes-corp-Executive-Salaries.html


In [4]:
#Makes a list of to make the graph later on
col=[('CEO Name', []), ('CEO Pay', []), ('Year', [])]

for n in range(10):
    currName = companies.at[n, 'Security']
    url='https://www1.salary.com/' + currName + '-Executive-Salaries.html'
    #url='https://www1.salary.com/3M-CO-Executive-Salaries.html'
    print(url)
    #using Beautiful Soup to read the contents of the website
    uClient = uReq(url)
    page_html = uClient.read()
    uClient.close()
    #html parser
    page_soup = soup(page_html, "html.parser")
    #grabs the divs that contain all the names, positions, and salaries
    containers = page_soup.findAll("div", {"class":"border-bottom font-semibold"})
    
    maxNameSoFar = ''
    maxPaySoFar = '0'
    maxPaySoFar = int(maxPaySoFar)

    #this will find the data for the ceo by going through the relevant rows
    for i in range(len(containers)):
        #this is to find the name of the person in the row
        name_container = containers[i].div.a.text
        #this is to find the position the person held
        pos_container = containers[i].find("div", "padding-bottom10 text-size15")
        #this is to find their salary
        sal = containers[i].find_all('div',class_='padding-left15')[1].div.text
        #this part converts the salary to a number
        sal = sal.strip()
        sal = sal.replace(r'$', '')
        sal = sal.replace(r',', '')
        try:
            sal=int(sal)
        except:
            print(sal)
            print('the ceo pay is not a number')
            break
        try:
            print(sal)
            #this tries to verify that the person in the row is the ceo
            if "CEO" in pos_container.text or "Chief Executive Officer" in pos_container.text:
                print('ceo')
                #If there are multiple CEOs, I'll take the one with the highest salary
                if sal > maxPaySoFar:
                    maxNameSoFar = name_container
                    maxPaySoFar = sal
                    
        except:
            pass
    col[0][1].append(maxNameSoFar)
    col[1][1].append(maxPaySoFar)
    col[2][1].append(2019)

https://www1.salary.com/3M-CO-Executive-Salaries.html


In [5]:
#making the dataframe
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [6]:
#displaying the dataframe
df

,CEO Name,CEO Pay,Year
0,,0,2019
